In [1]:
import random
import math

NumEval = 0    # Total number of evaluations
#조합의 문제. combinatiory? problem. numerical problem이랑은 달라
#delta만 없다

def main():
    # Create an instance of TSP
    p = createProblem()    # 'p': (numCities, locations, table)
    #txt 내용들이 tuple로 저장.
    # Call the search algorithm
    solution, minimum = steepestAscent(p)
    # Show the problem and algorithm settings
    describeProblem(p)
    displaySetting()
    # Report results
    displayResult(solution, minimum)
    
def createProblem():
    ## Read in a TSP (# of cities, locatioins) from a file.
    ## Then, create a problem instance and return it.
    fileName = "problem/tsp" + input("Enter the filename of function:") + ".txt"
    infile = open(fileName, 'r')
    # First line is number of cities
    numCities = int(infile.readline()) #도시수가 제일 처음에 있잖아 txt에
    locations = [] #좌표들
    line = infile.readline()  # The rest of the lines are locations #line: 스트링이다
    while line != '':
        locations.append(eval(line)) # Make a tuple and append #한줄읽은 string을 그대로 실행시켜줌. 
        #tuple로 location에 추가. eval없이 (line)만 하면 그냥 String만 입력될겨 한줄로 쭉. 
        #자동으로 숫자로 또 바꿔줘
        line = infile.readline()
    infile.close()
    table = calcDistanceTable(numCities, locations)
    return numCities, locations, table


def calcDistanceTable(numCities, locations): ### 구현해라! #거리계산
    table = []  #2d
    
    for i in range(numCities):
        row = [] #돌때마다 한줄이 초기화. 리스트 안에 리스트로.             
        
        for j in range(numCities): #tsp30.txt 이걸 표로 봐
            dx = locations[i][0] - locations[j][0]
            dy = locations[i][1] - locations[j][1]
            d = round(math.sqrt(dx**2 + dy**2),1) #제곱의 루트 math.sqrt. 소수첫째자리까지 반올림.
            row.append(d)        
        table.append(row)
    
    #table은 각각 도시간의 거리를 구해서 넣는거. 
    
    return table # A symmetric matrix of pairwise distances


def steepestAscent(p): #처음 한번 초기 tsp30을 한번 섞어주고 시작. 할때마다 좀 다르게 해야지
    current = randomInit(p)   # 'current' is a list of city ids
    valueC = evaluate(current, p)
    while True:
        neighbors = mutants(current, p) #후보자들 뽑아내서
        (successor, valueS) = bestOf(neighbors, p) #제일좋은거 찾고
        if valueS >= valueC:
            break
        else:
            current = successor
            valueC = valueS
    return current, valueC

def randomInit(p):   # Return a random initial tour
    n = p[0] #도시수
    init = list(range(n)) #0부터 29까지 리스트가 만들어지고
    random.shuffle(init) #섞어 (그냥 리스트를 섞어)
    return init


def evaluate(current, p): ### 현재 조합에서 순서대로 읽으면서 서로 인접한것끼리 거리 계산해서 더하기. 반복문 한번만.
    ## Calculate the tour cost of 'current'
    ## 'p' is a Problem instance
    ## 'current' is a list of city ids
    
    global NumEval
    NumEval += 1 
    
    n = p[0] #도시수
    table = p[2] #아까 distance 계산한거 2번째에 넣었잖아
    cost = 0
       
    for i in range(n-1):
        locFrom = current[i] #현재, 랜덤으로 섞인 table들의 거리 계산을 위해 좌표
        locTo = current[i+1] 
        cost += table[locFrom][locTo] #2d테이블 거리들 다 더해
    cost += table[current[n-1]][current[0]] #다 돈 다음에 처음으로 다시 돌아가야 하니까.                        
    return cost


def mutants(current, p): # Apply inversion #도시조합. 수치문제(n)처럼 못푼다. (delta  + 0.0001 더할거야?)
    #차라리 도시 방문 순서를 섞는거.
    # 5 0 3 8 9 2 1 ... --- 이렇게 위치 거리들 뽑힌거에서(current 였지?)
    # 5 0 9 8 3 2 1 ... --- 이렇게 3 8 9 를 뽑아서 앞뒤로 바꿔. 
    n = p[0] #도시 개수
    neighbors = []
    count = 0
    triedPairs = []
    while count <= n:  # Pick two random loci for inversion #도시수만큼 조합 뽑아내. 3이랑 9의 시작점과 끝점을 정해야지. 랜덤 인덱스 2개.
        i, j = sorted([random.randrange(n) for _ in range(2)]) #i = 2, j = 4겠네. 리스트 컴프리헨션. 리스트 내부에 값들을 한줄로 지정.
        #랜덤하게 2개의 값을 뽑아냈다. 0부터 n-1까지 값을 2번 뽑아내겠다. range(2) 가 반복문으로.
        if i < j and [i, j] not in triedPairs: #중복되지 않게
            triedPairs.append([i, j]) #같은거 나왔는지 겹치지 않게.
            curCopy = inversion(current, i, j) #뒤집는다. 3 8 9 -> 9 8 3 으로.
            count += 1
            neighbors.append(curCopy) #후보를 30개 뽑아내서 넣었네 반복문 돌면서.
    return neighbors

def inversion(current, i, j):  ## Perform inversion
    curCopy = current[:] #새로운 리스트를 만들어 주는거. 원본은 남겨두고.
    #curCopy = current 이것과의 차이? new 랑 같은건가?
    while i < j: #꼭! 자바에선 temp로 
        curCopy[i], curCopy[j] = curCopy[j], curCopy[i]
        i += 1
        j -= 1
    return curCopy

def bestOf(neighbors, p): ###
    best = neighbors[0]
    bestValue = evaluate(best,p) ## cost잖아.
    
    for i in range(1, len(neighbors)):
        newValue = evaluate(neighbors[i],p) 
        if newValue < bestValue: 
            best = neighbors[i]
            bestValue = newValue
    
    return best, bestValue

def describeProblem(p):
    print()
    n = p[0]
    print("Number of cities:", n)
    print("City locations:")
    locations = p[1]
    for i in range(n):
        print("{0:>12}".format(str(locations[i])), end = '')
        if i % 5 == 4:
            print()

def displaySetting():
    print()
    print("Search algorithm: Steepest-Ascent Hill Climbing")

def displayResult(solution, minimum):
    print()
    print("Best order of visits:")
    tenPerRow(solution)       # Print 10 cities per row
    print("Minimum tour cost: {0:,}".format(round(minimum)))
    print()
    print("Total number of evaluations: {0:,}".format(NumEval))

def tenPerRow(solution):
    for i in range(len(solution)):
        print("{0:>5}".format(solution[i]), end='')
        if i % 10 == 9:
            print()

main()





Number of cities: 100
City locations:
     (49, 3)    (74, 73)    (65, 36)    (39, 41)    (61, 99)
    (69, 44)    (88, 92)    (97, 28)    (53, 64)    (30, 77)
    (96, 62)    (61, 45)     (30, 3)    (66, 41)     (18, 9)
    (61, 64)    (28, 88)     (2, 72)    (80, 66)    (56, 38)
    (51, 16)     (18, 2)    (89, 18)    (67, 66)     (72, 6)
    (53, 32)    (29, 25)    (77, 69)    (89, 56)    (68, 88)
    (98, 53)    (36, 25)     (16, 0)    (20, 32)   (100, 10)
    (49, 49)    (85, 38)    (42, 52)     (3, 85)    (62, 77)
    (97, 87)    (75, 54)    (40, 19)    (32, 33)     (59, 1)
    (90, 43)    (62, 11)    (77, 14)    (88, 66)    (39, 32)
    (34, 69)    (12, 73)    (58, 88)    (34, 19)    (32, 45)
    (36, 36)    (84, 47)    (28, 18)    (23, 57)    (14, 52)
    (29, 38)     (0, 17)    (87, 96)    (61, 11)    (45, 56)
     (2, 60)    (97, 67)    (73, 70)    (49, 94)    (88, 55)
    (40, 55)    (23, 27)    (33, 68)    (70, 84)     (20, 0)
    (29, 59)    (35, 18)    (31, 77)    (66, 1